#Importing Libraries

In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import Subset, DataLoader, random_split
import os
import random
import pytorch_lightning as pl
import wandb
from pytorch_lightning.loggers import WandbLogger

#Importing Dataset

In [ ]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

#Data Pre-processing

In [ ]:
def load_and_split_data(directory, validation_ratio=0.2):
    #Load the full dataset
    full_dataset = datasets.ImageFolder(root=directory)

    #Get indices for each class
    targets = torch.tensor(full_dataset.targets)
    classes = full_dataset.classes
    class_to_indices = {class_name: torch.where(targets == class_index)[0]
                        for class_index, class_name in enumerate(classes)}

    #Initialize the lists for train and validation indices
    train_indices = []
    val_indices = []

    #Seed the random number generator for reproducibility
    random.seed(52)

    #Split indices for each class
    for indices in class_to_indices.values():
        indices = indices.tolist()
        random.shuffle(indices)  #Randomly shuffle the indices
        split = int(len(indices) * (1 - validation_ratio))  #Calculate split point
        train_indices += indices[:split]  #Indices for training
        val_indices += indices[split:]  #Indices for validation

    # Create subsets for training and validation
    training_data = Subset(full_dataset, train_indices)
    validation_data = Subset(full_dataset, val_indices)

    return training_data, validation_data

In [ ]:
train_dataset, val_dataset = load_and_split_data(train_dir)

In [ ]:
#Checking equal representation of each class in validation set
val_class_counts = {category: 0 for category in categories}

for idx in val_dataset.indices:
    label = train_dataset.dataset.targets[idx]  # get the label of each validation data point
    label_name = train_dataset.dataset.classes[label]  # convert label to class name
    val_class_counts[label_name] += 1

val_class_counts

{'Amphibia': 200,
 'Animalia': 200,
 'Arachnida': 200,
 'Aves': 200,
 'Fungi': 200,
 'Insecta': 200,
 'Mammalia': 200,
 'Mollusca': 200,
 'Plantae': 200,
 'Reptilia': 200}

In [ ]:
def initialize_data_loaders(train_dataset, val_dataset, augment_data, batch_size=64, resize_dim=224):
    """
    Prepares and returns DataLoader instances for training and validation data sets.

    Args:
    - train_dataset: Dataset for training.
    - val_dataset: Dataset for validation.
    - augment_data: String indicating if data augmentation ('Yes') should be applied to training data.
    - batch_size: Number of samples per batch.
    - resize_dim: Target size for each image dimension.

    Returns:
    - Training and validation DataLoader instances.
    """

    #Define data preprocessing steps
    training_preprocess = transforms.Compose([
        transforms.RandomResizedCrop(resize_dim) if augment_data == "Yes" else transforms.Resize((resize_dim, resize_dim)),
        transforms.RandomHorizontalFlip() if augment_data == "Yes" else transforms.Lambda(lambda img: img),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    validation_preprocess = transforms.Compose([
        transforms.Resize((resize_dim, resize_dim)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    #Assign transformations to datasets
    train_dataset.dataset.transform = training_preprocess
    val_dataset.dataset.transform = validation_preprocess

    #Initialize DataLoader for both datasets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader

In [ ]:
#Initialize DataLoader for test dataset
def create_test_loader(test_dir, batch_size=64, resize_dim=224):
    test_transforms = transforms.Compose([
        transforms.Resize((resize_dim, resize_dim)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return test_loader

In [ ]:
# Create Test DataLoader
test_loader = create_test_loader(test_dir)

# CNN Model

In [ ]:
class ImageClassifier(pl.LightningModule):

    def __init__(self, learning_rate, activation_type, use_batch_norm, augment_data, layer_filters, dropout_rate, filter_size, dense_neurons):
        super(ImageClassifier, self).__init__()
        self.val_losses = [] #To keep track of validation loss
        self.train_losses = []  #To keep track of training loss

        self.lr = learning_rate #Learning rate
        self.activation_type = activation_type  #Acticvation function to be used
        # self.use_batch_norm = use_batch_norm #Batch Normalization
        self.use_batch_norm = True if use_batch_norm == "Yes" else False
        self.augment_data = augment_data #Data Augmentation
        self.layer_filters = layer_filters #Number of layers in each filter
        self.dropout_rate = dropout_rate #Dropouts
        self.filter_size = filter_size  #Size of filter/Kernel
        self.dense_neurons = dense_neurons #Number of neurons in dense layer
        self.define_hyperparameters()

        self.conv_layers = nn.ModuleList()
        self.pool_layers = nn.ModuleList()
        self.batch_norm_layers = nn.ModuleList()
        self.choose_activation()
        # Input channels for iNaturalist images (RGB)
        in_channels = 3
        # Construct Convolutional Blocks
        for out_channels in layer_filters:
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, filter_size, padding=0))
            self.pool_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            if use_batch_norm:
                self.batch_norm_layers.append(nn.BatchNorm2d(out_channels))
            else:
                self.batch_norm_layers.append(nn.Identity())
            in_channels = out_channels  # Update in_channels for the next layer

        conv_output_size = 224 #Image size considered to be 224x224
        for _ in self.layer_filters:
            # Stride=1 and padding=0 for convolution layers, by default.
            conv_output_size = (conv_output_size - self.filter_size + 1)  # Convolution effect
            conv_output_size = conv_output_size // 2  # Pooling effect simplification

        self.flatten = nn.Flatten()
        flat_features = conv_output_size * conv_output_size * self.layer_filters[-1]
        self.dense_layer = nn.Linear(flat_features, self.dense_neurons)

        # Correct batch normalization after the dense layer if enabled
        if self.use_batch_norm:
            self.batch_norm_dense = nn.BatchNorm1d(self.dense_neurons)
        else:
            self.batch_norm_dense = nn.Identity()

        # The output layer that will contain 10 neurons for the 10 classes
        self.output_layer = nn.Linear(self.dense_neurons, 10)

        self.dropout = nn.Dropout(self.dropout_rate)

    #Forward Propagation
    def _forward_features(self, x):
        # Pass input through convolutional blocks
        for conv_layer, bn_layer, pool_layer in zip(self.conv_layers, self.batch_norm_layers, self.pool_layers):
            x = conv_layer(x)
            x = self.activation(x)
            x = bn_layer(x)
            x = pool_layer(x)
        return x

    def forward(self, x):
        x = self._forward_features(x)
        x = self.flatten(x)
        x = self.dense_layer(x)
        x = self.batch_norm_dense(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

    #Activation Functions
    def choose_activation(self):
        activations = {
            "ReLU": nn.ReLU(),
            "GELU": nn.GELU(),
            "SiLU": nn.SiLU(),
            "Mish": nn.Mish()
        }
        self.activation = activations.get(self.activation_type, nn.ReLU())

    def define_hyperparameters(self):
        self.save_hyperparameters()

    #Training loss and accuracy logging
    def training_step(self, train_batch):
        inputs, targets = train_batch
        predictions = self(inputs)
        loss = F.cross_entropy(predictions, targets)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        acc = (predictions.argmax(dim=1) == targets).float().mean()
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #Test loss and accuracy logging
    def validation_step(self, val_batch):
        inputs, targets = val_batch
        predictions = self(inputs)
        loss = F.cross_entropy(predictions, targets)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        acc = (predictions.argmax(dim=1) == targets).float().mean()
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #Test loss and accuracy logging
    def test_step(self, test_batch):
        inputs, targets = test_batch
        predictions = self(inputs)
        loss = F.cross_entropy(predictions, targets)
        self.log('test_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        acc = (predictions.argmax(dim=1) == targets).float().mean()
        self.log('test_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #Using optimizer function
    def configure_optimizers(self):
       return torch.optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)

    #Logging results to wandb and pytorch lightning
    def on_train_epoch_end(self):
        validation_result = self.trainer.callback_metrics.get('val_loss'), self.trainer.callback_metrics.get('val_acc')
        training_result = self.trainer.callback_metrics.get('train_loss'), self.trainer.callback_metrics.get('train_acc')

        # Optionally, log the epoch number as well
        wandb.log({"val_accuracy": validation_result[1].item(),
                  'val_loss': validation_result[0].item(),
                   'train_accuracy': training_result[1].item(),
                   'train_loss': training_result[0].item(),
                   'epoch': self.current_epoch})


In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'dropout_rate': {"values": [0.2, 0.3]},
        "activation_type": {
              "values": [ "ReLU", "SiLU", "GELU", "Mish"]
          },

          "learning_rate": {
              "values": [1e-3, 1e-4]
          }
          ,
        'layer_filters': {
            'values': [
                [64, 64, 64, 64, 64],
                [32, 32, 32, 32, 32],
                [16, 32, 64, 128, 256],
                [256, 128, 64, 32, 16],
                [128, 64, 32, 64, 128]
            ]
        },

        "augment_data":{
            "values":["Yes","No"]
        },
        "filter_size":{
            "values":[3,5]
        },
        "use_batch_norm":{
            "values":["Yes","No"]
        },
        "dense_neurons":{
            "values":[64, 128]
        },
          "epochs": {
              "values": [5,8]
          },
    }
}

In [ ]:
def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        config = wandb.config

        # Initialize your model with hyperparameters from wandb
        model = ImageClassifier(
            learning_rate=config.learning_rate,
            activation_type=config.activation_type,
            use_batch_norm=config.use_batch_norm,
            augment_data=config.augment_data,
            layer_filters=config.layer_filters,
            dropout_rate=config.dropout_rate,
            filter_size=config.filter_size,
            dense_neurons = config.dense_neurons
        )
        wandb_logger = WandbLogger()

        # Assume `train_dataloader` and `val_dataloader` are already defined
        trainer = pl.Trainer(max_epochs=config.epochs, accelerator="gpu", devices=1)
        # trainer = pl.Trainer(max_epochs=config.epochs, accelerator="cpu")
        # trainer = pl.Trainer(max_epochs=config.epochs)
        train_loader, val_loader = initialize_data_loaders(train_dataset, val_dataset, augment_data = config.augment_data)
        trainer.fit(model, train_loader, val_loader)

        # Log final validation accuracy
        run_name = f"lr_{config.learning_rate}_act_{config.activation_type}_bn_{config.use_batch_norm}_augment_{config.augment_data}_lf_{config.layer_filters}_kernel_{config.filter_size}_dp_{config.dropout_rate}_dn_{config.dense_neurons}_epoch_{config.epochs}"
        wandb.run.name = run_name
        # wandb.run.save()


In [ ]:
!wandb login "90512e34eff1bdddba0f301797228f7b64f546fc"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id = wandb.sweep(sweep_config,entity="cs22s027", project="DL_Assignment_2")

Create sweep with ID: u0v6tplb
Sweep URL: https://wandb.ai/cs22s027/DL_Assignment_2/sweeps/u0v6tplb


In [ ]:
wandb.agent('6h540hdw',function=train, count = 80)

wandb: Agent Starting Run: 576b0tmm with config:
wandb: 	activation_type: SiLU
wandb: 	augment_data: Yes
wandb: 	dense_neurons: 128
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 8
wandb: 	filter_size: 3
wandb: 	layer_filters: [128, 64, 32, 64, 128]
wandb: 	learning_rate: 0.001
wandb: 	use_batch_norm: No
wandb: Currently logged in as: cs22s027. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type       | Params
-------------------------------------------------
0 | conv_layers       | ModuleList | 188 K 
1 | pool_layers       | ModuleList | 0     
2 | batch_norm_layers | ModuleList | 832   
3 | activation        | SiLU       | 0     
4 | flatten           | Flatten    | 0     
5 | dense_layer       | Linear     | 409 K 
6 | batch_norm_dense  | Identity   | 0     
7 | output_layer      | Linear     | 1.3 K 
8 | dropout           | Dropout    | 0     
-------------------------------------------------
600 K

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()

In [ ]:
model = ImageClassifier(learning_rate = 0.001, activation_type = "Mish", use_batch_norm = "No",augment_data = "Yes", layer_filters = [128,64,32,64,128], dropout_rate= 0.3, filter_size = 3, dense_neurons = 128)
trainer = pl.Trainer(max_epochs=8, accelerator="gpu", devices=1)
train_loader, val_loader = initialize_data_loaders(train_dataset, val_dataset, augment_data = "Yes")
trainer.fit(model=model,train_dataloaders=train_loader,val_dataloaders=val_loader)
test_results = trainer.test(model,  test_loader)
if test_results:
    # Assuming you have one test DataLoader, thus one dictionary in test_results
    test_loss = test_results[0]["test_loss"]
    test_acc = test_results[0]["test_acc"] * 100  # Convert proportion to percentage if needed

    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")

In [ ]:
wandb.init(project='test_image_classification_', entity='cs22s027')

In [ ]:
import numpy as np
from torchvision.io import read_image
import matplotlib.pyplot as plt
from PIL import Image

# Define the categories and the path to the test directory
categories = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
test_dir='inaturalist_12K/val/'

# Prepare the data
x, y, y_pred = [], [], []

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize the images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model.eval()  # Ensure model is in evaluation mode

# Sampling 3 images per category and predicting
for category in categories:
    path = os.path.join(test_dir, category)
    images = os.listdir(path)
    sampled_images = random.sample(images, 3)  # Randomly sample 3 images
    for img_name in sampled_images:
        img_path = os.path.join(path, img_name)
        image = Image.open(img_path).convert('RGB')
        image_tensor = transform(image)
        x.append(image_tensor.numpy())  # Store the original for visualization
        y.append(category)
        image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            prediction = model(image_tensor)
            predicted_index = prediction.argmax(dim=1).item()
            y_pred.append(categories[predicted_index])

# Plotting the images and their predictions
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(10, 20))
for i, (img, true_label, pred_label) in enumerate(zip(x, y, y_pred)):
    ax = axes[i//3, i%3]
    img = np.transpose(img, (1, 2, 0))  # Change from CxHxW to HxWxC
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Unnormalize
    img = np.clip(img, 0, 1)
    ax.imshow(img)
    ax.set_title(f'Predicted: {pred_label}')  # Title on top as the predicted label
    ax.text(-0.1, 0.5, f'True: {true_label}', fontsize=12, transform=ax.transAxes,
            verticalalignment='center', horizontalalignment='right', rotation=90)
    ax.axis('off')

plt.tight_layout()
# Log the figure to wandb
#wandb.log({"Confusion Matrix" : [wandb.Image(plot_confusion_matrix(labels, y_pred, y_test),caption=f"Confusion Matrix with test accuracy = {accuracy} and loss = {loss}")]})
wandb.log({"Sampled Images with Predictions": [wandb.Image(fig, caption=f"Overall test accuracy = {test_acc:.2f}% and test loss = {test_loss:.4f}")]})

# Close the plot
plt.close(fig)

# Finish the run
wandb.finish()
